<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-08 11:21:38
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  9.00 K
Current:  1.37 C
-------------------
Today PnL: -39.36 K (-0.29%)
Current PnL: -13.54 L (-9.43%)
CY Booked + Current PnL: -6.58 L (-4.58%)
-------------------
Total profit:  4.65 L
Total loss:  -18.18 L
-------------------
Total Booked + Current PnL: 20.40 L (17.51%)
Total Booked PnL: 33.94 L (29.12%)
Curr Year Booked PnL: 6.96 L (5.1%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 72.01 L (52.72%)
Deployed:  1.17 C
Current:  1.37 C
CAGR/XIRR %: 10.7%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,APOLLOHOSP,6904.43,8285.00,-16.19,H-LC,91.67,114165.0,10599.0,10115.0,-0.07,10.23,8.86,20.00,32.0,1.05,0.84,25.75,X40N,BTT,HEALTHCARE
39,HINDALCO,651.95,761.55,-20.73,H-LC,83.33,109816.0,5504.0,12036.0,-0.83,5.28,10.96,16.81,27.0,0.46,0.80,22.17,X5K,ATH,METALS
88,UNIONBANK,123.87,163.00,-1.39,M-LC,25.00,171596.0,30756.0,13728.0,-0.82,21.84,8.00,31.59,88.0,2.24,1.26,49.06,XY24,NTT,BANKS
92,VIPIND,488.80,489.00,-211.87,H-SC,97.92,80672.0,-14644.0,14682.0,-0.26,-15.36,18.20,0.04,157.0,-1.00,0.59,59.12,OX40N,NTT,MISC
71,SBIN,760.30,863.00,-11.51,M-LC,33.33,213855.0,12376.0,14842.0,0.02,6.14,6.94,13.51,77.0,0.83,1.57,19.51,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,116.91,190.00,-31.29,M-MC,8.33,179562.0,-246.0,112657.0,-0.82,-0.14,62.74,62.52,173.0,-0.00,1.32,33.90,XR,NTT,BANKS
60,NATIONALUM,189.63,247.44,-50.15,H-MC,54.69,99921.0,-583.0,31225.0,-0.02,-0.58,31.25,30.49,97.0,-0.02,0.73,31.64,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,84.88,NaN,NaN,51.04,18589.0,-31004.0,0.0,3.93,-62.52,0.00,-62.52,270.0,-inf,0.14,24.66,XY24,ATH,MISC
85,TRIDENT,37.35,48.00,26.76,M-SC,80.21,79781.0,-12474.0,38782.0,3.79,-13.52,48.61,28.51,233.0,-0.32,0.58,36.52,XR,NTT,TEXTILES
32,ENRIN,1377.95,3091.10,NaN,NaN,95.83,77278.0,42829.0,0.0,3.03,124.33,0.00,124.33,274.0,inf,0.57,18.17,AR,ATH,ELECTRICAL
52,JPPOWER,18.73,26.20,-12.60,L-SC,100.00,176072.0,34230.0,22344.0,2.65,24.13,12.69,39.88,262.0,1.53,1.29,78.43,XY24,NTT,POWER
13,BATAINDIA,1550.24,2096.00,-30.15,M-SC,72.92,104339.0,-24331.0,69625.0,2.34,-18.91,66.73,35.20,205.0,-0.35,0.76,6.45,X40,NTT,FOOTWEAR


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,RELAXO,902.64,1176.0,-30.47,H-SC,92.71,62899.0,-52639.0,87631.0,-2.95,-45.56,139.32,30.28,131.0,-0.60,0.46,22.88,X40N,NTT,FOOTWEAR
24,COFFEEDAY,59.14,80.0,-57.75,L-SC,47.92,63629.0,-49920.0,89971.0,-2.76,-43.96,141.40,35.27,265.0,-0.55,0.47,55.73,XR,NTT,HOTELS
0,5PAISA,564.75,565.0,154.81,H-SC,29.17,137315.0,-49617.0,49694.0,-2.04,-26.54,36.19,0.04,149.0,-1.00,1.01,31.36,OX40N,NTT,FINANCE
22,CIPLA,1495.00,1795.0,-22.84,H-LC,31.25,99234.0,-931.0,21028.0,-1.95,-0.93,21.19,20.07,30.0,-0.04,0.73,6.96,X40N,BTT,PHARMA
11,BANDHANBNK,214.25,400.0,-32.36,H-SC,9.38,201979.0,-46551.0,262027.0,-1.90,-18.73,129.73,86.70,134.0,-0.18,1.48,33.62,XY24,NTT,BANKS


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,45.86,M-SC,22.92,127035.0,-15444.0,16070.0,0.11,-10.84,12.65,0.44,247.0,-0.96,0.93,49.94,OX40N,NTT,BANKS
79,SYMPHONY,1306.42,1306.0,-29.18,M-SC,27.08,150807.0,-20334.0,20284.0,-0.03,-11.88,13.45,-0.03,191.0,-1.00,1.10,8.39,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,94.79,148974.0,-25585.0,25624.0,0.05,-14.66,17.20,0.02,133.0,-1.00,1.09,29.18,OX40N,NTT,PAINTS
92,VIPIND,488.80,489.0,-211.87,H-SC,97.92,80672.0,-14644.0,14682.0,-0.26,-15.36,18.20,0.04,157.0,-1.00,0.59,59.12,OX40N,NTT,MISC
81,TATAELXSI,7332.28,7332.0,-18.32,X-MC,6.25,85687.0,-16965.0,16957.0,-0.88,-16.53,19.79,-0.00,59.0,-1.00,0.63,30.25,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TCS,3794.03,4998.00,-18.38,X-LC,41.67,265863.0,-30071.0,123972.0,-0.09,-10.16,46.63,31.73,1.0,-0.24,1.95,6.41,X40,BTT,IT
48,INFY,1461.46,2275.00,-9.19,X-LC,75.00,291144.0,29543.0,116079.0,-0.03,11.29,39.87,55.67,2.0,0.25,2.13,18.09,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-14.11,X-LC,62.50,269192.0,-1155.0,35776.0,-0.29,-0.43,13.29,12.81,5.0,-0.03,1.97,12.58,XY25,NTT,FMCG
17,BRITANNIA,4983.33,6446.05,17.58,X-LC,71.88,269514.0,40281.0,27005.0,-0.42,17.57,10.02,29.35,10.0,1.49,1.97,28.06,XY25,ATH,FMCG
74,SIEMENS,4406.90,7969.85,-2.21,X-LC,28.12,82600.0,-27572.0,116648.0,0.79,-25.03,141.22,80.85,11.0,-0.24,0.61,21.66,AR,ATH,ELECTRICAL


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3091.10,NaN,NaN,95.83,77278.0,42829.0,0.0,3.03,124.33,0.00,124.33,274.0,inf,0.57,18.17,AR,ATH,ELECTRICAL
28,DIGITIDE,188.38,237.72,NaN,NaN,42.71,50812.0,9557.0,1250.0,1.54,23.17,2.46,26.19,272.0,7.65,0.37,19.63,XY24,ATH,IT
52,JPPOWER,18.73,26.20,-12.60,L-SC,100.00,176072.0,34230.0,22344.0,2.65,24.13,12.69,39.88,262.0,1.53,1.29,78.43,XY24,NTT,POWER
88,UNIONBANK,123.87,163.00,-1.39,M-LC,25.00,171596.0,30756.0,13728.0,-0.82,21.84,8.00,31.59,88.0,2.24,1.26,49.06,XY24,NTT,BANKS


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,92.75,M-MC,50.00,231178.0,6216.0,161432.0,0.58,2.76,69.83,74.53,189.0,0.04,1.69,34.22,XY24,BTT,STEEL
54,KPIGREEN,497.21,731.97,24.30,H-SC,32.29,127386.0,2089.0,57069.0,-1.45,1.67,44.80,47.22,143.0,0.04,0.93,59.37,MH,ATH,POWER
20,CAMS,3643.00,5250.99,2.09,H-SC,18.75,114688.0,12684.0,32342.0,-1.11,12.43,28.20,44.14,126.0,0.39,0.84,30.70,X40N,ATH,MISC
88,UNIONBANK,123.87,163.00,-1.39,M-LC,25.00,171596.0,30756.0,13728.0,-0.82,21.84,8.00,31.59,88.0,2.24,1.26,49.06,XY24,NTT,BANKS
95,WIPRO,243.46,420.00,-3.31,M-LC,89.58,165943.0,14998.0,94455.0,-0.04,9.94,56.92,72.51,71.0,0.16,1.22,14.31,XR,NTT,IT


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3091.10,NaN,NaN,95.83,77278.0,42829.0,0.0,3.03,124.33,0.00,124.33,274.0,inf,0.57,18.17,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-38.94,M-SC,58.33,201346.0,29725.0,132345.0,-0.23,17.32,65.73,94.43,201.0,0.22,1.48,48.61,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-3.31,M-LC,89.58,165943.0,14998.0,94455.0,-0.04,9.94,56.92,72.51,71.0,0.16,1.22,14.31,XR,NTT,IT
45,INDIAMART,2327.09,4850.92,-48.45,H-SC,66.67,134408.0,11072.0,122688.0,-0.09,8.98,91.28,108.45,123.0,0.09,0.98,33.34,AR,ATH,MISC
54,KPIGREEN,497.21,731.97,24.30,H-SC,32.29,127386.0,2089.0,57069.0,-1.45,1.67,44.80,47.22,143.0,0.04,0.93,59.37,MH,ATH,POWER


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,84.88,NaN,NaN,51.04,18589.0,-31004.0,0.0,3.93,-62.52,0.00,-62.52,270.0,-inf,0.14,24.66,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.00,-57.75,L-SC,47.92,63629.0,-49920.0,89971.0,-2.76,-43.96,141.40,35.27,265.0,-0.55,0.47,55.73,XR,NTT,HOTELS
32,ENRIN,1377.95,3091.10,NaN,NaN,95.83,77278.0,42829.0,0.0,3.03,124.33,0.00,124.33,274.0,inf,0.57,18.17,AR,ATH,ELECTRICAL
58,MASFIN,326.60,399.50,-17.80,H-SC,73.96,93600.0,-4380.0,26245.0,0.92,-4.47,28.04,22.32,145.0,-0.17,0.69,35.75,XR,ATH,FINANCE
9,BAJAJFINSV,1994.80,2758.00,-1.80,X-LC,39.58,98054.0,309.0,37084.0,-0.26,0.32,37.82,38.26,18.0,0.01,0.72,30.93,X40,BTT,FINANCE


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-12.60,L-SC,100.00,176072.0,34230.0,22344.0,2.65,24.13,12.69,39.88,262.0,1.53,1.29,78.43,XY24,NTT,POWER
3,ALKYLAMINE,2347.98,4546.37,8.94,X-SC,98.96,101390.0,427.0,94100.0,-1.67,0.42,92.81,93.63,67.0,0.00,0.74,51.02,SR,ATH,CHEMICALS
92,VIPIND,488.80,489.00,-211.87,H-SC,97.92,80672.0,-14644.0,14682.0,-0.26,-15.36,18.20,0.04,157.0,-1.00,0.59,59.12,OX40N,NTT,MISC
15,BLUESTARCO,1646.70,2337.55,-3.74,H-SC,96.88,180530.0,15860.0,53220.0,-1.17,9.63,29.48,41.95,124.0,0.30,1.32,17.82,X40N,ATH,AC
32,ENRIN,1377.95,3091.10,NaN,NaN,95.83,77278.0,42829.0,0.0,3.03,124.33,0.00,124.33,274.0,inf,0.57,18.17,AR,ATH,ELECTRICAL


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.92
1,25,41.87
2,50,70.54


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.44
LC    33.96
MC    21.49
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.91
X40      12.95
XY25     12.04
X40N     11.90
XR       10.07
AR        8.18
OX40N     8.18
MH        1.66
X5K       1.44
X200      1.42
SR        1.22
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.42
M-SC    15.90
H-LC    14.11
X-LC    12.93
H-MC     9.49
X-MC     7.59
M-LC     5.88
X-SC     4.44
M-MC     4.05
L-SC     2.68
L-LC     1.04
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.20,-0.76,34.45
IT,12.11,-9.96,63.96
FINANCE,8.49,-12.59,53.45
BANKS,7.71,-8.49,52.77
MISC,5.83,-20.69,76.86
PAINTS,5.70,-12.21,34.33
ELECTRICAL,5.49,1.49,44.91
HEALTHCARE,4.63,-1.97,31.75
AUTO,4.30,-15.08,65.68


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2667211.0,26
XR,1086182.0,13
AR,1029643.0,10
X40,687078.0,10
X40N,539790.0,11
XY25,474778.0,8
OX40N,312984.0,11
SR,176795.0,2
MH,88294.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1993999.0,20
M-SC,1768391.0,21
H-LC,615420.0,12
X-LC,544547.0,8
H-MC,537790.0,8
M-MC,416825.0,3
X-MC,416651.0,6
X-SC,312128.0,5
M-LC,259609.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      893047.0      6
M-SC       XY24      810617.0      7
H-SC       AR        526842.0      3
M-SC       XR        334806.0      4
M-MC       XY24      304168.0      2
H-MC       XY24      302730.0      3
X-LC       X40       277135.0      3
H-SC       XR        214940.0      3
           X40N      212101.0      4
H-LC       X40N      197665.0      4
M-SC       AR        184706.0      2
           OX40N     164473.0      5
X-MC       X40       156003.0      2
H-LC       X40       155758.0      3
L-SC       XR        151931.0      2
M-LC       XY24      150312.0      2
X-SC       XR        123472.0      1
M-SC       XY25      121469.0      1
X-LC       AR        116648.0      1
M-MC       XR        112657.0      1
X-MC       XY25      104304.0      1
X-LC       XY25      102538.0      3
M-LC       XR         94455.0      1
X-SC       SR         94100.0      1
H-SC       OX40N      90000.0      3
X-MC       XY24       86700.0      1
H-MC       XY25       83840.0      1
M-SC       SR         82695.0      1
H-LC       AR         80880.0      1
           X5K        70921.0      2
M-SC       X40        69625.0      1
H-LC       X200       67155.0      1
H-MC       AR         62327.0      1
L-SC       AR         58240.0      1
H-SC       MH         57069.0      1
L-MC       XR         53921.0      1
X-SC       XY24       53002.0      1
X-MC       X40N       52687.0      1
X-LC       X40N       48226.0      1
L-LC       XY25       47785.0      1
H-LC       XY24       43041.0      1
X-SC       OX40N      41554.0      2
H-MC       MH         31225.0      1
           X40N       29111.0      1
           X40        28557.0      1
L-SC       XY24       22344.0      1
X-MC       OX40N      16957.0      1
M-LC       XY25       14842.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
